In [15]:
# App to be run once in a while (~1 every 2 weeks or month) and display on the screen list of upcoming birthdays
# a few days/weeks ahead of time

# Concept:
# google docs stores a list of birth dates in "Birthday List" sheet file
# script opens the list every x days (as scheduled by chron job) and checks if any birthdays are upcoming 
# Displays a single window and message on the screen with a list of these people
# Code from chatGPT!

import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta
from pync import Notifier
import time

# Google Sheets authentication
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/tomwojcik/Desktop/Personal Docs (recent)/Programming/BdayReminderApp Project/gentle-mapper-424019-s2-3cfaf34bd16f.json', scope)
client = gspread.authorize(creds)

# Open the Google Sheet
sheet = client.open("Birthday List").sheet1

# Fetch all records
records = sheet.get_all_records()

# Calculate upcoming birthdays
num_days = 30   # how many days ahead you'd like to check
today = datetime.now().date()
upcoming_days = num_days  # Number of days ahead to check for birthdays
upcoming_birthdays = []

for record in records:
    name = record.get("FirstName")
    birth_date_str = record.get("Birthdate")
    birth_date = datetime.strptime(birth_date_str, "%m-%d").date()
    next_birthday = birth_date.replace(year=today.year)
    
    if next_birthday < today:
        next_birthday = next_birthday.replace(year=today.year + 1)
    
    if 0 <= (next_birthday - today).days <= upcoming_days:
        upcoming_birthdays.append(f"{name} ({next_birthday.strftime('%m-%d')})")
        
# Function to display notification repeatedly
def show_notification(message, title, interval, duration):
    start_time = time.time()
    while (time.time() - start_time) < duration:
        Notifier.notify(message, title=title)
        time.sleep(interval)
        
    # Display notification
if upcoming_birthdays:
    message = "Upcoming Birthdays:\n" + "\n".join(upcoming_birthdays)
    show_notification(message, "Birthday Reminder", interval=5, duration=15)  # Notify every 5 seconds for 15 sec
else:
    show_notification("No upcoming birthdays in the next 30 days.", "Birthday Reminder", interval=5, duration=10)
    
    # To-do improvements:
# 1. match days and message
# 2. setup as a cron job? or another way to running automatically once every x days
    
    
# Future extensions:
# 1. Auto prepare a simple message for each person, and creates a draft g-mail to be sent
# 2. the message is prepped by an LLM, to make it more interesting
# 3. the message ads a LLM-generated image based on a prompt to make it fun and personable 
#      ... potentially with a link to person's photo
# 4. the message is personalized by a system that searches for info and interactions about a person (web, fb, email, files?)
#      ... and creates a message based on that (RAG-like?)